In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point

In [6]:
df_full = pd.read_csv("data/zomato_df_final_data.csv")

In [4]:
gdf = geopandas.read_file("data/sydney.geojson")

In [5]:
df.columns

Index(['address', 'cost', 'cuisine', 'lat', 'link', 'lng', 'phone',
       'rating_number', 'rating_text', 'subzone', 'title', 'type', 'votes',
       'groupon', 'color', 'cost_2', 'cuisine_color'],
      dtype='object')

In [7]:
# haven't decided what to do with cuisine
df = df_full.drop(columns = ["address", "link", "phone", "title"], axis = 1)

In [25]:
df.dtypes

cost             float64
cuisine           object
lat              float64
lng              float64
rating_number    float64
subzone           object
type              object
votes            float64
groupon            int64
cost_2           float64
address           object
dtype: object

In [10]:
# for regression 1 we wont use rating_text
df = df.drop(columns = ["rating_text"], axis = 1)

In [12]:
len(df.cuisine_color.unique())
# not sure why we have this

5

In [14]:
# do some feature engineering
df.groupon = df.groupon.astype(int)

In [17]:
df = df.drop(columns = ["color", "cuisine_color"], axis = 1)

In [46]:
df.isna().sum()

cost              346
cuisine             0
lat               192
lng               192
rating_number    3316
subzone             0
type               48
votes            3316
groupon             0
cost_2            346
address             0
dtype: int64

In [24]:
# bringing address to see in helps
df["address"] = df_full["address"]

In [45]:
temp = df[df.lat.isna()]
temp.address

156       Level 4 & 5, Citymark, 683-689 George Street,...
317       Level 3, North End, QVB, 455 George Street, \...
438       The Cannery, Shop 1/34 Morley Avenue, Roseber...
468       Shop 5, 302 Church Street (enter via Phillip ...
573                    Lower Ground, 55 York Street, \tCBD
                               ...                        
10387     Ground Level, Forestville Centre, Starkey Str...
10477     Food Court, Greenway Plaza, 210 Church Street...
10483                  14 Bunlunga Lane, Mona Vale, Sydney
10491     Shop 2, Coogee Village Shopping Centre, 184 C...
10499            136 Sailors Bay Road, Northbridge, Sydney
Name: address, Length: 192, dtype: object

In [36]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [42]:
# i = 0
# for address in temp.address : 
#     print(f"{i}. address : {address}")
#     i += 1
#     try : 
#         geolocator = Nominatim(user_agent="address_to_ssc_code")
#         location = geolocator.geocode(address)
#         if location:
#             latitude, longitude = location.latitude, location.longitude
#             print(f"Latitude: {latitude}, Longitude: {longitude}")
#         else:
#             print("Address not found.")
#     except GeocoderTimedOut:
#         print("Geocoding service timed out. Address not found.")
#     except Exception as e:
#         print(f"An error occurred: Max retries exceeded")

In [60]:
missing_suburbs = temp.address.apply(lambda x : x.split(",")[-1])
# missing_suburbs = [suburb.strip() for suburb in missing_suburbs]

In [61]:
missing_suburbs.unique()

array([' Chinatown', ' \tCBD', ' Sydney',
       '         Westfield Main Entrance', ' Randwick', ' 2 Kingsway',
       ' \tChinatown', 'then turn left)', ' WATERLOO NSW',
       ' 2010. (Entry via Campbell St (65).)', ' \tCastle Hill',
       ' Pirrama Park Cafe\nPyrmont', ' CBD', ' Chancellery Walk',
       ' \tPyrmont', ' University of New South Wales Kensington',
       ' North Sydney', ' Sydney Casula'], dtype=object)